In [10]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")
STOP_WORDS_ES = stopwords.words("spanish")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luiscarlosmarrufopadilla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Datos base de grasas Interlub
df_raw = pd.read_csv("../../datos/FINAL/datos_grasas_Tec_limpio.csv", encoding="utf-8")

# Datos de grasas de la competencia, los dejamos listos para después
#df_competencia = pd.read_csv("../../datos/FINAL/datos_pdfs_f.csv", encoding="utf-8")

In [12]:
print(df_raw.head())

   idDatosGrasas codigoGrasa     Aceite Base                        Espesante  \
0              1     Grasa_1  Semi-Sintetico     Complejo Sulfonato de Calcio   
1              2     Grasa_2      Mineral HT     Complejo Sulfonato de Calcio   
2              3     Grasa_3      Mineral HT     Complejo Sulfonato de Calcio   
3              4     Grasa_4      Mineral HT     Complejo Sulfonato de Calcio   
4              5     Grasa_5      Mineral HT  Complejo de Aluminio - Poliurea   

   Grado NLGI Consistencia  Viscosidad del Aceite Base a 40°C. cSt  \
0                      2.0                                   680.0   
1                      1.5                                   460.0   
2                      2.0                                   460.0   
3                      2.0                                   220.0   
4                      1.5                                   680.0   

   Penetración de Cono a 25°C, 0.1mm  Punto de Gota, °C  \
0                              27

In [13]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   idDatosGrasas                             51 non-null     int64  
 1   codigoGrasa                               51 non-null     object 
 2   Aceite Base                               47 non-null     object 
 3   Espesante                                 45 non-null     object 
 4   Grado NLGI Consistencia                   51 non-null     float64
 5   Viscosidad del Aceite Base a 40°C. cSt    50 non-null     float64
 6   Penetración de Cono a 25°C, 0.1mm         51 non-null     float64
 7   Punto de Gota, °C                         51 non-null     int64  
 8   Estabilidad Mecánica, %                   48 non-null     float64
 9   Punto de Soldadura Cuatro Bolas, kgf      50 non-null     float64
 10  Desgaste Cuatro Bolas, mm               

In [14]:
df = df_raw.copy()

# Asegura formato Grasa_1, Grasa_2, ...
df["codigoGrasa"] = [f"Grasa_{i+1}" for i in range(len(df))]

cols_drop_features = [
    "idDatosGrasas",
    "Indice de Carga-Desgaste",
    "categoria",
]

df_v = df.drop(columns=cols_drop_features)
df_v.set_index("codigoGrasa", inplace=True)

print(df_v.head())


                Aceite Base                        Espesante  \
codigoGrasa                                                    
Grasa_1      Semi-Sintetico     Complejo Sulfonato de Calcio   
Grasa_2          Mineral HT     Complejo Sulfonato de Calcio   
Grasa_3          Mineral HT     Complejo Sulfonato de Calcio   
Grasa_4          Mineral HT     Complejo Sulfonato de Calcio   
Grasa_5          Mineral HT  Complejo de Aluminio - Poliurea   

             Grado NLGI Consistencia  Viscosidad del Aceite Base a 40°C. cSt  \
codigoGrasa                                                                    
Grasa_1                          2.0                                   680.0   
Grasa_2                          1.5                                   460.0   
Grasa_3                          2.0                                   460.0   
Grasa_4                          2.0                                   220.0   
Grasa_5                          1.5                                   

In [15]:
cols_texto = ["subtitulo", "descripcion", "beneficios", "aplicaciones"]

descripcion_grasas = (
    df_v[cols_texto + ["Registro NSF"]]
    .copy()
    .reset_index()  # que codigoGrasa sea una columna
)

# Registro NSF como binario (tiene certificado / no tiene)
descripcion_grasas["Registro NSF"] = (
    descripcion_grasas["Registro NSF"].notnull().astype(int)
)

print(descripcion_grasas.head())

  codigoGrasa                                          subtitulo  \
0     Grasa_1  Grasa de servicio pesado para alta resistencia...   
1     Grasa_2          Grasa para lubricaciÃ³n de equipo pesado.   
2     Grasa_3          Grasa para lubricaciÃ³n de equipo pesado.   
3     Grasa_4        Grasa lubricante para condiciones extremas.   
4     Grasa_5  Grasa lubricante para servicio pesado con alta...   

                                         descripcion  \
0  El producto es grasa lubricante de servicio pe...   
1  El producto es una grasa lubricante de gran ad...   
2  El producto es una grasa lubricante de gran ad...   
3  El producto es una grasa lubricante elaborada ...   
4  El producto es un grasa lubricante con propied...   

                                          beneficios  \
0  Excelentes caracterÃ­sticas de resistencia al ...   
1  Extremo soporte de carga.\r\n@Alto contenido d...   
2  Extremo soporte de carga.\r\n@Alto contenido d...   
3  Alta estabilidad tÃ©rmica y

In [16]:
for c in cols_texto:
    descripcion_grasas[c] = descripcion_grasas[c].fillna("")

descripcion_grasas["soup"] = (
    descripcion_grasas["subtitulo"] + " " +
    descripcion_grasas["descripcion"] + " " +
    descripcion_grasas["beneficios"] + " " +
    descripcion_grasas["aplicaciones"]
).str.strip()

print(descripcion_grasas[["codigoGrasa", "soup"]].head())

  codigoGrasa                                               soup
0     Grasa_1  Grasa de servicio pesado para alta resistencia...
1     Grasa_2  Grasa para lubricaciÃ³n de equipo pesado. El p...
2     Grasa_3  Grasa para lubricaciÃ³n de equipo pesado. El p...
3     Grasa_4  Grasa lubricante para condiciones extremas. El...
4     Grasa_5  Grasa lubricante para servicio pesado con alta...


In [17]:
df_features = df_v.copy()

# Registro NSF binario en features
df_features["Registro NSF"] = df_features["Registro NSF"].notnull().astype(int)

# Columnas que NO usaremos como features
cols_drop_extra = ["Corrosión al Cobre", "Factor de Velocidad"]
df_features = df_features.drop(columns=cols_drop_extra)

# One hot para categóricas importantes
df_features = pd.get_dummies(
    df_features,
    columns=["Aceite Base", "Espesante", "color", "textura"],
    drop_first=False
)

# Faltantes como -99 (consistente con lo que ya usabas)
df_features = df_features.fillna(-99.0)

print(df_features.head())

             Grado NLGI Consistencia  Viscosidad del Aceite Base a 40°C. cSt  \
codigoGrasa                                                                    
Grasa_1                          2.0                                   680.0   
Grasa_2                          1.5                                   460.0   
Grasa_3                          2.0                                   460.0   
Grasa_4                          2.0                                   220.0   
Grasa_5                          1.5                                   680.0   

             Penetración de Cono a 25°C, 0.1mm  Punto de Gota, °C  \
codigoGrasa                                                         
Grasa_1                                  279.0                304   
Grasa_2                                  300.0                304   
Grasa_3                                  280.0                300   
Grasa_4                                  281.0                300   
Grasa_5                  

In [18]:
tfidf = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1, 4),
    min_df=1,
    stop_words=STOP_WORDS_ES,
)

tfidf_matrix = tfidf.fit_transform(descripcion_grasas["soup"])
cosine_sim_text = cosine_similarity(tfidf_matrix)

tfidf_matrix.shape, cosine_sim_text.shape

((51, 4163), (51, 51))

In [19]:
numeric_cols = df_features.select_dtypes(include="number").columns.tolist()

df_numeric = df_features[numeric_cols].copy()

# Aquí df_numeric ya no debería tener NaN, pero por si acaso:
df_numeric = df_numeric.fillna(df_numeric.median())

scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(df_numeric)

cosine_sim_numeric = cosine_similarity(numeric_scaled)

cosine_sim_numeric.shape

(51, 51)

In [20]:
peso_texto = 0.7
peso_numerico = 0.3

cosine_sim_hybrid = (
    peso_texto * cosine_sim_text +
    peso_numerico * cosine_sim_numeric
)

cosine_sim_hybrid.shape


(51, 51)

In [21]:
codigos = df_features.index.to_list()
indices = pd.Series(range(len(codigos)), index=codigos)
codigos[:5], indices.head()


(['Grasa_1', 'Grasa_2', 'Grasa_3', 'Grasa_4', 'Grasa_5'],
 Grasa_1    0
 Grasa_2    1
 Grasa_3    2
 Grasa_4    3
 Grasa_5    4
 dtype: int64)

In [22]:
def recomendar_grasas(
    codigo_grasa,
    top_n=5,
    modo="hibrido",     # "texto", "numerico" o "hibrido"
    peso_texto=0.7,
    peso_numerico=0.3,
):
    """
    Devuelve un Top-N de grasas similares a `codigo_grasa`.
    
    modo:
      - "texto": solo similitud por descripciones (soup)
      - "numerico": solo propiedades técnicas
      - "hibrido": combinación texto + numérico
    """
    if codigo_grasa not in indices:
        raise ValueError(f"{codigo_grasa} no existe en la base.")

    idx = indices[codigo_grasa]

    if modo == "texto":
        sim_vec = cosine_sim_text[idx]
    elif modo == "numerico":
        sim_vec = cosine_sim_numeric[idx]
    else:
        sim_vec = (
            peso_texto * cosine_sim_text[idx] +
            peso_numerico * cosine_sim_numeric[idx]
        )

    # Enumerar y ordenar por score de mayor a menor
    sim_scores = list(enumerate(sim_vec))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Quitar la misma grasa (idx) y quedarnos con Top N
    sim_scores = [s for s in sim_scores if s[0] != idx][:top_n]

    indices_top = [s[0] for s in sim_scores]
    scores_top = [s[1] for s in sim_scores]

    # Armar tabla de salida
    result = (
        df_features.iloc[indices_top]
        .reset_index()
        .rename(columns={"index": "codigoGrasa"})
    )

    result["similaridad"] = np.round(scores_top, 3)

    # Agregar subtítulo para interpretar la recomendación
    result = result.merge(
        descripcion_grasas[["codigoGrasa", "subtitulo"]],
        on="codigoGrasa",
        how="left"
    )

    # Columnas más importantes que podrías mostrar en el dashboard
    cols_salida = [
        "codigoGrasa",
        "similaridad",
        "subtitulo",
        "Temperatura de Servicio °C, min",
        "Temperatura de Servicio °C, max",
        "Grado NLGI Consistencia",
        "Viscosidad del Aceite Base a 40°C. cSt",
    ]

    # Filtra solo columnas que existan (por si cambias algo después)
    cols_salida = [c for c in cols_salida if c in result.columns]

    return result[cols_salida]


In [23]:
print(recomendar_grasas("Grasa_10", top_n=5, modo="hibrido"))


  codigoGrasa  similaridad  Temperatura de Servicio °C, min  \
0     Grasa_2        0.997                              -30   
1     Grasa_9        0.994                              -30   
2     Grasa_8        0.880                              -30   
3    Grasa_11        0.877                               -8   
4    Grasa_12        0.736                                5   

   Temperatura de Servicio °C, max  Grado NLGI Consistencia  \
0                              150                      1.5   
1                              150                      1.0   
2                              150                      0.0   
3                              150                      2.0   
4                              150                      3.0   

   Viscosidad del Aceite Base a 40°C. cSt  
0                                   460.0  
1                                   800.0  
2                                   800.0  
3                                   800.0  
4                     

In [24]:
print(recomendar_grasas("Grasa_10", top_n=5, modo="texto"))

  codigoGrasa  similaridad  Temperatura de Servicio °C, min  \
0     Grasa_2          1.0                              -30   
1     Grasa_3          1.0                              -10   
2     Grasa_8          1.0                              -30   
3     Grasa_9          1.0                              -30   
4    Grasa_11          1.0                               -8   

   Temperatura de Servicio °C, max  Grado NLGI Consistencia  \
0                              150                      1.5   
1                              150                      2.0   
2                              150                      0.0   
3                              150                      1.0   
4                              150                      2.0   

   Viscosidad del Aceite Base a 40°C. cSt  
0                                   460.0  
1                                   460.0  
2                                   800.0  
3                                   800.0  
4                     

In [25]:
print(recomendar_grasas("Grasa_10", top_n=5, modo="numerico"))

  codigoGrasa  similaridad  Temperatura de Servicio °C, min  \
0     Grasa_2        0.989                              -30   
1     Grasa_9        0.979                              -30   
2    Grasa_43        0.768                              -15   
3     Grasa_8        0.599                              -30   
4    Grasa_11        0.588                               -8   

   Temperatura de Servicio °C, max  Grado NLGI Consistencia  \
0                              150                      1.5   
1                              150                      1.0   
2                              150                      2.0   
3                              150                      0.0   
4                              150                      2.0   

   Viscosidad del Aceite Base a 40°C. cSt  
0                                   460.0  
1                                   800.0  
2                                   320.0  
3                                   800.0  
4                     

In [26]:
import pandas as pd

def recomendar_grasas(
    grado_nlgi=None,        # int o lista, ej. 2 o [1,2]
    temp_min_trabajo=None,  # temperatura mínima de operación que necesitas (°C)
    temp_max_trabajo=None,  # temperatura máxima de operación que necesitas (°C)
    carga_timken_min=None,  # carga mínima Timken (lb)
    requiere_nsf=None,      # None = no filtra, 1 = sí requiere, 0 = explícitamente sin NSF
    top_n=5                 # cuántas opciones regresar
):
    # Partimos del df original (el que leíste del CSV)
    base = df.copy()

    # Máscara booleana para ir filtrando
    mask = pd.Series(True, index=base.index)

    # --- Filtros duros por condiciones ---
    if grado_nlgi is not None:
        if isinstance(grado_nlgi, (list, tuple, set)):
            mask &= base["Grado NLGI Consistencia"].isin(grado_nlgi)
        else:
            mask &= base["Grado NLGI Consistencia"] == grado_nlgi

    # La grasa debe soportar una temperatura mínima igual o más baja que la que necesitas
    if temp_min_trabajo is not None:
        mask &= base["Temperatura de Servicio °C, min"] <= temp_min_trabajo

    # Y una temperatura máxima igual o mayor que la que necesitas
    if temp_max_trabajo is not None:
        mask &= base["Temperatura de Servicio °C, max"] >= temp_max_trabajo

    if carga_timken_min is not None:
        mask &= base["Carga Timken Ok, lb"] >= carga_timken_min

    # Tratamiento de Registro NSF usando la regla:
    # "si no viene el identificador, asumimos que no tiene certificado"
    if requiere_nsf is not None:
        if int(requiere_nsf) == 1:
            mask &= base["Registro NSF"].notnull()
        else:  # requiere_nsf == 0
            mask &= base["Registro NSF"].isna()

    candidatos = base.loc[mask].copy()

    if candidatos.empty:
        print("No se encontraron grasas que cumplan todas las condiciones.")
        return candidatos

    # --- Ranking opcional: ordenar por lo cerca que están de la temp_max_trabajo ---
    if temp_max_trabajo is not None:
        candidatos["score_temp"] = (
            candidatos["Temperatura de Servicio °C, max"] - temp_max_trabajo
        ).abs()
        candidatos = candidatos.sort_values("score_temp")

    # Qué columnas quieres ver en la recomendación
    columnas_mostrar = [
        "codigoGrasa",
        "subtitulo",
        "Aceite Base",
        "Espesante",
        "Grado NLGI Consistencia",
        "Temperatura de Servicio °C, min",
        "Temperatura de Servicio °C, max",
        "Punto de Gota, °C",
        "Carga Timken Ok, lb",
        "Resistencia al Lavado por Agua a 80°C, %",
        "Registro NSF",
    ]

    # Filtrar solo columnas que existan (por si alguna falta en tus datos)
    columnas_mostrar = [c for c in columnas_mostrar if c in candidatos.columns]

    return candidatos[columnas_mostrar].head(top_n)

Quiero una grasa NLGI 2, que funcione de −20°C a 180°C, con Carga Timken ≥ 40 lb:

In [27]:
print(recomendar_grasas(
    grado_nlgi=2,
    temp_min_trabajo=-20,
    temp_max_trabajo=180,
    carga_timken_min=40,
    requiere_nsf=None,
    top_n=5
))

   codigoGrasa                                          subtitulo  \
13    Grasa_14  Grasa de servicio pesado para alta temperatura...   

         Aceite Base                     Espesante  Grado NLGI Consistencia  \
13  Mineral Refinado  Complejo Sulfonato de Calcio                      2.0   

    Temperatura de Servicio °C, min  Temperatura de Servicio °C, max  \
13                              -20                              190   

    Punto de Gota, °C  Carga Timken Ok, lb  \
13                304                 60.0   

    Resistencia al Lavado por Agua a 80°C, %  Registro NSF  
13                                       0.4           NaN  


Quiero grasas grado 1 o 2, con NSF, para equipo alimenticio, que llegue al menos a 150°C:

In [28]:
print(recomendar_grasas(
    grado_nlgi=[1, 2],
    temp_min_trabajo=None,
    temp_max_trabajo=150,
    carga_timken_min=None,
    requiere_nsf=1,
    top_n=10
))


   codigoGrasa                                          subtitulo  \
47    Grasa_48  Grasa especial para alta velocidad en la indus...   
49    Grasa_50  Grasa especial para la industria alimenticia y...   
16    Grasa_17  Grasa de alta resistencia para industria alime...   
26    Grasa_27  Grasa de alta resistencia para industria alime...   
27    Grasa_28  Grasa de alta resistencia para industria alime...   

       Aceite Base                     Espesante  Grado NLGI Consistencia  \
47     Mineral USP                           NaN                      2.0   
49  Semi-Sintetico                     Bentonita                      1.0   
16     Mineral USP          Complejo de Aluminio                      2.0   
26     Mineral USP  Complejo Sulfonato de Calcio                      1.0   
27     Mineral USP  Complejo Sulfonato de Calcio                      2.0   

    Temperatura de Servicio °C, min  Temperatura de Servicio °C, max  \
47                              -10               

In [29]:
df_competencia = pd.read_csv("../../datos/FINAL/datos_pdfs_f.csv", encoding="utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 31-32: invalid continuation byte